In [0]:
# mount ref https://adb-984752964297111.11.azuredatabricks.net/editor/notebooks/1242762840109196?o=984752964297111#command/1242762840109197 

In [0]:
mnts = dbutils.fs.mounts()

mnts_df = spark.createDataFrame(mnts)
cv_test_mount = mnts_df.filter(mnts_df.mountPoint.contains("hlscvproject"))
display(cv_test_mount)

In [0]:
display(dbutils.fs.ls("/mnt/hlscvproject"))

In [0]:
# Create the directory using dbutils.fs.mkdirs
dbutils.fs.mkdirs("/mnt/hlscvproject/test_folder/")

In [0]:
# !mkdir dbfs:/mnt/hlscvproject/test_folder/
!cp /Workspace/Users/may.merkletan@databricks.com/db_CV/test_mmt.txt /dbfs/mnt/hlscvproject/test_folder/test_mmt.txt

In [0]:
dbutils.fs.mkdirs("/mnt/hlscvproject/test_folder2/")

In [0]:
display(dbutils.fs.ls("/mnt/hlscvproject"))

In [0]:
display(dbutils.fs.ls("/mnt/hlscvproject/test_folder/"))

In [0]:
!ls -lah /dbfs/mnt/hlscvproject/test_folder/

In [0]:
### IAM assignment is required but disabled for external location credentials (Azure connector mmtdevs) on azure databricks 

## testing oneenv-adls as external location credential instead of mmtdevs  

In [0]:
# Listing the contents of the target directory to verify the file copy
display(dbutils.fs.ls("dbfs:/Volumes/mmt_mlops_demos/cv/cv-test"))

In [0]:
# Copying the file using dbutils.fs.cp
dbutils.fs.cp("/Workspace/Users/may.merkletan@databricks.com/db_CV/test.txt", 
              "/Volumes/mmt_mlops_demos/cv/cv-test/test.txt")

In [0]:
!ls -lah /Volumes/mmt_mlops_demos/cv/cv-test/

# cp: cannot create regular file '/Volumes/mmt_mlops_demos/cv/cv-test/test.txt': Permission denied
# cp: cannot create regular file '/Volumes/mmt_mlops_demos/cv/cv-test/test_folder2/test_mmt.txt': No such file or directory 

In [0]:
!ls -lah /Volumes/mmt_mlops_demos/cv/cv-test/

In [0]:
### TEST with new container cv-test2 on mmtdevs storage account + oneenv-adls [Service Principal -- legacy?] credential (instead of ManagedID) in the field-eng-east workspace

# https://adb-984752964297111.11.azuredatabricks.net/explore/credentials/oneenv-adls?o=984752964297111 
# --> added cv-test2
# https://adb-984752964297111.11.azuredatabricks.net/explore/locations/cv-test2?o=984752964297111

# https://portal.azure.com/#view/Microsoft_AAD_IAM/ManagedAppMenuBlade/~/Overview/objectId/19dc1fa8-21bd-48d0-82e5-3f959870683e/appId/ed573937-9c53-4ed6-b016-929e765443eb



### NB there is a oneenv-adls storage account but it is located in USWEST! which is not ideal for field-eng-east USEAST2 workspace
# https://portal.azure.com/#@DataBricksInc.onmicrosoft.com/resource/subscriptions/3f2e4d32-8e8d-46d6-82bc-5bb8d962328b/resourceGroups/oneenv/providers/Microsoft.Storage/storageAccounts/oneenvadls/overview

In [0]:
!ls -lah /Volumes/mmt_mlops_demos/cv/cv-test2/

In [0]:
!cp /Workspace/Users/may.merkletan@databricks.com/db_CV/test_mmt.txt /Volumes/mmt_mlops_demos/cv/cv-test2/test_mmt.txt
##WORKS?!

In [0]:
!ls /Volumes/mmt_mlops_demos/cv/cv-test2/

In [0]:
!ls -lah /Volumes/mmt_mlops_demos/cv/

In [0]:
!mkdir /Volumes/mmt_mlops_demos/cv/cv-test2/test_folder

In [0]:
!cp /Workspace/Users/may.merkletan@databricks.com/db_CV/test_mmt.txt /Volumes/mmt_mlops_demos/cv/cv-test2/test_folder/test2_mmt.txt

In [0]:
dbutils.fs.put("/Volumes/mmt_mlops_demos/cv/cv-test2/test_file.txt", "This is a test file.")

In [0]:
dbutils.fs.ls("/Volumes/mmt_mlops_demos/cv/cv-test2/")

In [0]:
mnts = dbutils.fs.mounts()
display(mnts)

In [0]:
mnts_df = spark.createDataFrame(mnts)
cv_test_mount = mnts_df.filter(mnts_df.mountPoint.contains("hls_cv"))
display(cv_test_mount)

In [0]:
dbutils.secrets.list('mmt')

In [0]:
dbutils.secrets.get('mmt','hls_cv_azSAS_mmtdevs_cv-test2')

In [0]:
sas_token = dbutils.secrets.get('mmt','hls_cv_azSAS_mmtdevs_cv-test2')

storage_account_name = "mmtdevs"
container_name = "cv-test2"

mount_name = "hls_cv"

dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    mount_point = f"/mnt/{mount_name}",
    extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token}
)

In [0]:
mnts = dbutils.fs.mounts()

mnts_df = spark.createDataFrame(mnts)
cv_test_mount = mnts_df.filter(mnts_df.mountPoint.contains("hls_cv"))
display(cv_test_mount)

In [0]:
display(dbutils.fs.ls("/mnt/hls_cv/"))

In [0]:
# dbutils.fs.rm("/dbfs/mnt/hls_cv/test_put_str2dbfs_mnt.txt", True)

In [0]:
dbutils.fs.put("dbfs:/mnt/hls_cv/test_put_str2dbfs_mnt.txt", "This is a test file.")

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/hls_cv/"))

In [0]:
!cat /dbfs/mnt/hls_cv/test_put_str2dbfs_mnt.txt

In [0]:
# configs = {
#     "fs.azure.account.auth.type": "OAuth",
#     "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#     "fs.azure.account.oauth2.client.id": "<application-id>",
#     "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="<scope-name>", key="<service-credential-key-name>"),
#     "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<directory-id>/oauth2/token"
# }

# dbutils.fs.mount(
#     source = "abfss://<container-name>@<storage-account-name>.dfs.core.windows.net/",
#     mount_point = "/mnt/<mount-name>",
#     extra_configs = configs
# )